In [1]:
import pandas as pd
import json
import geopandas as gpd
import os
%matplotlib inline
import geohash
import geopandas as gp
import pandas as pd
import math
import geojson
from geojson import MultiLineString
from shapely.geometry import MultiLineString
from shapely.geometry import MultiPoint
import numpy as np
from shapely import geometry
from shapely.geometry import Point, Polygon, box,LineString
from geopandas import datasets, GeoDataFrame, read_file
from geopandas.tools import overlay
from matplotlib import pyplot
import matplotlib.pyplot as plt
from shapely.geometry.polygon import LinearRing, Polygon
from numpy import cos,sin,arccos
import numpy as np
from pylab import *
from shapely.ops import cascaded_union
from shapely.geometry import Point, LineString, mapping, MultiPoint
import scipy.interpolate
import geopy
import geopy.distance
import time
import folium
import gmaps
import networkx as nx
from shapely.ops import nearest_points
import itertools

In [5]:
traffic_flow = gpd.read_file("D:/Datasets/MIMOS/traffic_flow_26082020")
print(len(traffic_flow))
traffic_flow = traffic_flow.head(1000)
print(len(traffic_flow))

353400
1000


In [6]:
traffic_flow.to_file('D:/Datasets/MIMOS/test_intersection/test_intersection.shp')

In [7]:
gadm = gpd.read_file('D:/Datasets/GeoSpatial/gadm_data/gadm36_MYS_1.shp')

In [16]:
gadm.head()

,GID_0,NAME_0,GID_1,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry
0,MYS,Malaysia,MYS.1_1,Johor,Johor Darul Takzim|Johore,None,Negeri,State,None,MY.JH,"MULTIPOLYGON (((104.11916 1.92278, 104.11806 1..."
1,MYS,Malaysia,MYS.2_1,Kedah,Kedah Darul Aman,None,Negeri,State,None,MY.KH,"MULTIPOLYGON (((100.32889 5.66444, 100.32917 5..."
2,MYS,Malaysia,MYS.3_1,Kelantan,<Null>,None,Negeri,State,None,MY.KN,"MULTIPOLYGON (((102.17395 6.20126, 102.17395 6..."
3,MYS,Malaysia,MYS.4_1,Kuala Lumpur,Federal Territory of Kuala Lumpur,None,Wilayah Persekutuan,Federal Territory,None,MY.KL,"POLYGON ((101.73227 3.03781, 101.70284 3.03007..."
4,MYS,Malaysia,MYS.5_1,Labuan,Federal Territory of Labuan,None,Wilayah Persekutuan,Federal Territory,None,MY.LA,"MULTIPOLYGON (((115.14328 5.18610, 115.13979 5..."


In [18]:
kl_selangor_gadm = gadm[gadm['NAME_1'].isin(['Kuala Lumpur', 'Selangor'])]

In [19]:
kl_selangor_gadm.head()

,GID_0,NAME_0,GID_1,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry
3,MYS,Malaysia,MYS.4_1,Kuala Lumpur,Federal Territory of Kuala Lumpur,None,Wilayah Persekutuan,Federal Territory,None,MY.KL,"POLYGON ((101.73227 3.03781, 101.70284 3.03007..."
14,MYS,Malaysia,MYS.15_1,Selangor,Selangor Darul Ehsan,None,Negeri,State,None,MY.SL,"MULTIPOLYGON (((101.76600 2.86978, 101.74870 2..."


In [20]:
traffic_flow.head()

,TMC_PC,TMC_DE,TMC_QD,TMC_LE,FC,CF_SP,CF_SU,CF_FF,CF_JF,CF_CN,PBT,DE,geometry
0,4069,P2/Jalan Gambang,-,0.73129,2,63.00,63.00,63.9,0.18779,0.70,2020-08-26T07:59:35Z,3,"LINESTRING (103.22677 3.77076, 103.22693 3.771..."
1,4070,Jalan Lapangan Terbang,-,0.40270,2,44.00,44.00,73.5,2.90000,0.70,2020-08-26T07:59:35Z,3,"LINESTRING (103.22693 3.77117, 103.22709 3.772..."
2,4071,Permatang Badak Baru,-,5.23481,2,77.77,77.77,83.5,0.91569,0.71,2020-08-26T07:59:35Z,3,"LINESTRING (103.22747 3.77477, 103.22764 3.775..."
3,4072,Persiaran Sultan Abu Bakar,-,3.63231,2,80.00,80.00,88.8,1.32132,0.70,2020-08-26T07:59:35Z,3,"LINESTRING (103.26349 3.80117, 103.26369 3.801..."
4,4073,Jalan Kuantan Sungai Lembing,-,1.59160,2,82.00,82.00,90.0,1.18518,0.70,2020-08-26T07:59:35Z,3,"LINESTRING (103.28147 3.82830, 103.28182 3.828..."


In [24]:
import geopandas as gpd
#g1 = gpd.GeoDataFrame.from_file("crime_stat.shp")
#g2 = gpd.GeoDataFrame.from_file("population.shp")
data = []
for index, gadm in kl_selangor_gadm.iterrows():
    for index2, traffic in traffic_flow.iterrows():
        if gadm['geometry'].intersects(traffic['geometry']):
            data.append({'geometry': traffic['geometry'].intersection(gadm['geometry']),
                        'TMC_PC': traffic['TMC_PC'],
                        'TMC_DE': traffic['TMC_DE'],
                         'TMC_QD': traffic['TMC_QD'],
                        'FC': traffic['FC'],
                         'CF_SP': traffic['CF_SP'],
                         'CF_SU': traffic['CF_SU'],
                         'CF_FF': traffic['CF_FF'],
                         'CF_JF': traffic['CF_JF'],
                         'CF_CN': traffic['CF_CN'],
                         'PBT': traffic['PBT'],
                         'DE': traffic['DE']
                        })

df = gpd.GeoDataFrame(data,columns=['geometry', 'TMC_PC', 'TMC_DE','TMC_QD', 'TMC_LE', 'FC', 
                                    'CF_SP', 'CF_SU', 'CF_FF', 'CF_JF', 'CF_CN', 'PBT', 'DE' ])
#df.to_file('intersection.shp')
# control of the results in mi case, first values
df.head() # image from a Jupiter/IPython notebook

,geometry,TMC_PC,TMC_DE,TMC_QD,TMC_LE,FC,CF_SP,CF_SU,CF_FF,CF_JF,CF_CN,PBT,DE
0,"LINESTRING (101.56610 3.05161, 101.56549 3.051...",4712,E6/LEBUHRAYA UTARA SELATAN HUBUNGAN TENGAH (NO...,-,NaN,1,50.0,50.0,50.0,0.00000,0.7,2020-08-26T07:59:35Z,None
1,"LINESTRING (101.37124 3.01689, 101.37123 3.016...",4669,2/Jalan Parang/Jalan Kurau/Jalan Pelabuhan Utara,-,NaN,4,33.0,33.0,43.0,2.17054,0.7,2020-08-26T07:59:35Z,Lebuhraya Selat Klang
2,"LINESTRING (101.37113 3.01771, 101.37103 3.018...",4670,Lebuh Sultan Mohamed 1 (West),-,NaN,2,68.0,68.0,77.3,1.60413,0.7,2020-08-26T07:59:35Z,Lebuhraya Selat Klang
3,"LINESTRING (101.37842 3.03190, 101.37860 3.032...",4671,Jalan Sultan Abdul Samad 40,-,NaN,2,72.0,72.0,85.7,2.13146,0.7,2020-08-26T07:59:35Z,Lebuhraya Selat Klang
4,"LINESTRING (101.38230 3.03616, 101.38255 3.036...",4672,Lebuh Sultan Mohamed 1 (East),-,NaN,2,72.0,72.0,84.9,2.02591,0.7,2020-08-26T07:59:35Z,Lebuhraya Selat Klang


In [26]:
df.to_file('D:/Datasets/MIMOS/test_intersection_cut/test_intersection_cut.shp')